### Simple XGBoost example
(credit to Jason Brownlee's [blogpost](https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/))

In [2]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
#the raw data only has values, because of that the variable names have to
#be provided separately 
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi',
'age', 'class'] 
diabetes = pd.read_csv(url, names=names)
print(diabetes.shape)
array = diabetes.values
X = array[:,0:8]
Y = array[:,8]
seed = 7
test_size = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=test_size,
                                                    random_state=seed)
                                        
# fit model on training data
model = XGBClassifier()
model.fit(X_train, Y_train)


(768, 9)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
#accuracy = accuracy_score(Y_test, predictions)
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: {:.3f}".format(accuracy*100))

Accuracy: 77.953


/home/perro/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Notice that the accuracy score is only slightly better than what we obtained with a logistic regression in the [metrics notebook](./Misc/metrics.ipynb)!!

Better to calculate a cross-validation score:

In [19]:
from sklearn.model_selection import cross_val_score

results = cross_val_score(model, X, Y,scoring='accuracy')
print("Accuracy: {:.3f} ({:.3f})".format(results.mean()*100, results.std()*100))

Accuracy: 77.613 (3.075)


/home/perro/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/perro/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/perro/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Using the XGBoost library cross-validation function (not sure if the 'error' metric from the XGB library is equivalent to the accuracy metric from Scikit-learn):

In [11]:
import xgboost as xgb
xgb_param = model.get_xgb_params()#dict with all parameters passed
xgtrain = xgb.DMatrix(X, label=Y)#data structure used by XGBoost
#command below performs cross validation
cvresult = xgb.cv(xgb_param,xgtrain,
                  num_boost_round=model.get_params()['n_estimators'],
                  nfold=10,metrics='error',
                  early_stopping_rounds=50,
                  verbose_eval=True)

[0]	train-error:0.226567+0.0114547	test-error:0.265687+0.05025
[1]	train-error:0.216726+0.0144738	test-error:0.250102+0.0475936
[2]	train-error:0.217305+0.0113642	test-error:0.243609+0.046351
[3]	train-error:0.213109+0.0120741	test-error:0.243592+0.0451752
[4]	train-error:0.211083+0.010177	test-error:0.243592+0.0501304
[5]	train-error:0.206742+0.0106843	test-error:0.237081+0.0493064
[6]	train-error:0.201534+0.0102724	test-error:0.239696+0.0534899
[7]	train-error:0.202256+0.0106656	test-error:0.242293+0.0488374
[8]	train-error:0.198929+0.00993185	test-error:0.242259+0.0446438
[9]	train-error:0.197049+0.00845536	test-error:0.239627+0.0433144
[10]	train-error:0.194735+0.00869992	test-error:0.233151+0.0459395
[11]	train-error:0.196471+0.0090049	test-error:0.230536+0.0422137
[12]	train-error:0.195169+0.00953189	test-error:0.227922+0.0453629
[13]	train-error:0.193722+0.00869972	test-error:0.229221+0.0434253
[14]	train-error:0.192564+0.00907848	test-error:0.227922+0.0453629
[15]	train-error:0

In [9]:
print('XGB-CV result: {:.3f} ({:.3f})'.format(100*(1-0.239644),0.0533958))

XGB-CV result: 76.036 (0.053)
